<a href="https://colab.research.google.com/github/csabi0312/Credit-Card-Fraud-Detection-using-SMOTE-and-Deep-Neural-Network/blob/main/Fraud_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install keras_tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.3/167.3 KB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 44.1 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

from tensorflow.keras.callbacks import TensorBoard
from imblearn.over_sampling import SMOTE
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch
from tensorflow.keras.utils import plot_model
import sklearn
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, confusion_matrix
import seaborn as sns
from tensorflow.keras import optimizers
from tensorflow.keras import callbacks
from kerastuner.tuners import RandomSearch
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
from sklearn.metrics import classification_report, confusion_matrix


# Load the creditcard.csv file into a Pandas dataframe named 'data'
data = pd.read_csv("/content/drive/MyDrive/DataScience/Fraud_detection/creditcard.csv")

# Print the columns of the dataframe
print(data.columns)

# Split the data into training and testing sets, with a test size of 0.3 and random state of 22
data_train, data_test = train_test_split(data, random_state=22, test_size=0.3)

# Separate the features (X) and target variable (y) for both the training and testing sets
X_train = data_train.drop("Class", axis=1)
y_train = data_train["Class"].astype(int)
X_test = data_test.drop("Class", axis=1)
y_test = data_test["Class"].astype(int)

# Print the shapes of the training and testing data
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

# Calculate and print the ratio of anomalies (fraudulent transactions) in the training and testing sets
print("ratio of anomalies in the training dataset:", np.sum(y_train)/len(y_train))
print("ratio of anomalies in the testing dataset:", np.sum(y_test)/len(y_test))

# Scale the training and testing data using the StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Resample the training data using the Synthetic Minority Over-sampling Technique (SMOTE) to balance the classes
sm = SMOTE(random_state=42, k_neighbors=5)
X_res, y_res = sm.fit_resample(X_train, y_train)

# Calculate and print the ratio of anomalies (fraudulent transactions) in the resampled training data and original training data
print("ratio of anomalies in the resampled training dataset:", np.sum(y_res)/len(y_res))
print("ratio of anomalies in the original training dataset:", np.sum(y_train)/len(y_train))


def build_model(hp):
    model = keras.Sequential()

    # Add the input layer
    model.add(layers.Input(shape=(30,)))

    # Tune the number of hidden layers
    for i in range(hp.Int('num_layers', 3, 8)):
        # Tune the number of units in each hidden layer
        model.add(layers.Dense(units=hp.Int('units_' + str(i), min_value=32, max_value=512, step=32), activation='relu'))

    # Add the output layer
    model.add(layers.Dense(1, activation='sigmoid'))

    # Tune the learning rate of the optimizer
    lr = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    # Compile the model
    model.compile(optimizer=optimizers.Adam(learning_rate=lr), loss='binary_crossentropy', metrics=["accuracy"])

    return model

# Define the Keras Tuner
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=1,
    directory='my_dir',
    project_name='my_project',
    overwrite=True)

# Search for the best hyperparameters
tuner.search(X_res, y_res, epochs=5, validation_split=0.2)

# Get the best model
best_model = tuner.get_best_models(num_models=1)[0]



# Fit the best model to the training data
history = best_model.fit(X_res, y_res, epochs=50, batch_size=32, validation_split=0.2, callbacks=[callbacks.EarlyStopping(patience=15)])

# Test the best model on the test set
y_pred = best_model.predict(X_test)
y_pred_classes = (y_pred > 0.5).astype(int)
test_loss, test_acc = best_model.evaluate(X_test, y_test, verbose=2)

print("Test Accuracy:", test_acc)
print("Classification Report:")
print(classification_report(y_test, y_pred_classes))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_classes))

Trial 5 Complete [00h 03m 23s]
val_accuracy: 0.0

Best val_accuracy So Far: 1.0
Total elapsed time: 00h 15m 50s


Epoch 1/50
9951/9951 [==============================] - 31s 3ms/step - loss: 0.0040 - accuracy: 0.9992 - val_loss: 0.0098 - val_accuracy: 0.9963
Epoch 2/50
9951/9951 [==============================] - 29s 3ms/step - loss: 0.0026 - accuracy: 0.9995 - val_loss: 8.2434e-04 - val_accuracy: 1.0000
Epoch 3/50
9951/9951 [==============================] - 29s 3ms/step - loss: 0.0019 - accuracy: 0.9996 - val_loss: 2.4705e-04 - val_accuracy: 1.0000
Epoch 4/50
9951/9951 [==============================] - 29s 3ms/step - loss: 0.0015 - accuracy: 0.9997 - val_loss: 2.2620e-04 - val_accuracy: 1.0000
Epoch 5/50
9951/9951 [==============================] - 29s 3ms/step - loss: 0.0013 - accuracy: 0.9997 - val_loss: 2.3215e-04 - val_accuracy: 1.0000
Epoch 6/50
9951/9951 [==============================] - 29s 3ms/step - loss: 0.0010 - accuracy: 0.9998 - val_loss: 1.5671e-04 - val_accuracy: 1.0000
Epoch 7/50
9951/9951 [==============================] - 29s 3ms/step - loss: 8.6656e-04 - accuracy: 0.9998 - v

In [ ]:
plot_model(best_model, to_file='model.png', show_shapes=True, show_layer_names=True)

In [ ]:
# Extract the training and validation loss values from the history variable
train_loss = history.history['loss']
val_loss = history.history['val_loss']

# Create a dataframe with the loss values and their corresponding epochs
df = pd.DataFrame({'Epoch': range(len(train_loss)), 'Training Loss': train_loss, 'Validation Loss': val_loss})

# Use Seaborn to create a line plot of the training and validation loss
sns.lineplot(data=df, x='Epoch', y='Training Loss', label='Training Loss')
sns.lineplot(data=df, x='Epoch', y='Validation Loss', label='Validation Loss')

# Add axis labels and a title to the plot
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss over Epochs')